In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW>

In [3]:
from time import time
from preprocessing.Preprocessor import Preprocessor

In [4]:
pp = Preprocessor(spark)

In [5]:
train = pp.to_tfidf('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = True)
val =  pp.to_tfidf('Cleaned Data/val.csv')

In [6]:
#train = pp.balance_classes(train)

In [7]:
train.cache()
val.cache()

DataFrame[features: vector, label: int]

In [8]:
train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1854,[0,1,3,12,1...|    2|
|(1854,[3,13,74,89...|    0|
|(1854,[0,9,10,19,...|    0|
|(1854,[4,8,36,184...|    2|
|(1854,[2,5,6,17,2...|    2|
+--------------------+-----+
only showing top 5 rows



In [9]:
val.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1854,[8,14,16,21...|    0|
|(1854,[0,1,4,7,8,...|    2|
|(1854,[0,1,3,6,10...|    0|
|(1854,[0,2,3,6,7,...|    2|
|(1854,[0,1,2,4,5,...|    2|
+--------------------+-----+
only showing top 5 rows



In [10]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [11]:
"""lst_smooth = [0, 1.0, 100.0, 500, 1000]
lst_model = []
lst_model_transform = []


for i in lst_smooth:
    nb = NaiveBayes(smoothing=i, modelType="multinomial")
    model = nb.fit(train)
    lst_model.append(model)
    train_preds_and_labels = model.transform(train).select('prediction', 'label')
    lst_model_transform.append(train_preds_and_labels)"""
    
    
nb = NaiveBayes(smoothing = 1.0, modelType="multinomial")
model = nb.fit(train)
train_fitting_check = model.transform(train).select('prediction', 'label')

In [12]:
"""train_m = Metrics()
for i in lst_model_transform:
    train_m.f1(i)
    train_m.accuracy(i)
    train_m.confusion_matrix(i)
    train_m.performance(i)
    """

from metrics.metrics import Metrics
train_m = Metrics()
train_m.f1(train_fitting_check)
train_m.accuracy(train_fitting_check)
train_m.confusion_matrix(train_fitting_check)
train_m.performance(train_fitting_check)

F1 score for given DataFrame: 70.6

Accuracy on given DataFrame: 68.1%

                Predictions
            negative    neutral     positive    
negative    17843       4778        5726        
neutral     2621        4401        3300        
positive    8023        12119       55959       
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    62.6        62.9        62.8        
neutral     20.7        42.6        27.8        
positive    86.1        73.5        79.3        



In [13]:
#print(lst_model)

In [14]:
"""model_predictions = []

for i in lst_model:
    predictions = i.transform(val)
    model_predictions.append(predictions)"""
predictions = model.transform(val)

In [15]:
"""for i in model_predictions:
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
    accuracy = evaluator.evaluate(i)
    print("Test set accuracy = " + str(accuracy))"""

evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol="prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.6770328363669624


In [16]:
val_m = Metrics()
val_fitting_check = model.transform(val).select('prediction', 'label')
val_m.f1(val_fitting_check)
val_m.accuracy(val_fitting_check)
val_m.confusion_matrix(val_fitting_check)
val_m.performance(val_fitting_check)

F1 score for given DataFrame: 70.2

Accuracy on given DataFrame: 67.7%

                Predictions
            negative    neutral     positive    
negative    5110        1401        1756        
neutral     742         1146        927         
positive    2371        3396        15950       
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    62.1        61.8        62.0        
neutral     19.3        40.7        26.2        
positive    85.6        73.4        79.1        



In [17]:
"""train_m = Metrics()
train_m.f1(model_predictions[0])
train_m.f1(model_predictions[1])
train_m.f1(model_predictions[2])
train_m.f1(model_predictions[3])
train_m.f1(model_predictions[4])"""

'train_m = Metrics()\ntrain_m.f1(model_predictions[0])\ntrain_m.f1(model_predictions[1])\ntrain_m.f1(model_predictions[2])\ntrain_m.f1(model_predictions[3])\ntrain_m.f1(model_predictions[4])'